In [1]:
# prompt: import recordlinkage
# import pandas
# Or install if required

try:
  import recordlinkage
except ModuleNotFoundError:
  !pip install recordlinkage
  import recordlinkage

try:
  import pandas as pd
except ModuleNotFoundError:
  !pip install pandas
  import pandas as pd
import numpy as np

In [2]:
df_register = pd.read_excel('public_data_registerP.xlsx')
df_results = pd.read_excel('public_data_resultsP.xlsx')
df_anonymised = pd.read_csv('anonymised_dataP.csv')
df_survey = pd.read_csv('survey_listP.txt', sep='\t')
df_survey.columns = ['name']
df_register['dob'] = pd.to_datetime(df_register['dob'], format='%d-%m-%Y')
df_register['age'] = np.floor((pd.to_datetime('today') - df_register['dob']).dt.days / 365.25).astype(int)
df_anonymised = df_anonymised.rename(columns={"Age": "age"})

In [3]:
df_register.head()

,name,sex,dob,zip,citizenship,marital_status,last_voted,age
0,"Tellez, Jose",Male,2004-05-27,2100,Denmark,Never married,2,20
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Never married,1,20
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Never married,1,23
3,"el-Rad, Taaj",Male,1999-06-19,2400,Denmark,Never married,2,25
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Never married,1,22


In [4]:
# vote_mapping = {
#     0: 'In-person',  # Vote cast on paper
#     1: 'Online',     # Vote cast electronically
#     2: 'Abstained'   # Did not vote
# }

# df_register['evote'] = df_register['last_voted'].map(vote_mapping)

In [5]:
# Find the register rows that match the names in survey_listK
matched_register = df_register[df_register['name'].isin(df_survey['name'])]
matched_register

,name,sex,dob,zip,citizenship,marital_status,last_voted,age
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Never married,1,20
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Never married,1,23
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Never married,1,22
9,"Grant, Debray",Male,2003-12-03,2100,Denmark,Never married,1,20
10,"Blackmore, Gabriela",Female,2004-09-18,2100,Denmark,Never married,1,20
...,...,...,...,...,...,...,...,...
1447,"el-Shahin, Muzaina",Female,1926-11-02,2300,Denmark,Widowed,0,98
1454,"Herrera, Addison",Female,1966-07-05,2300,Denmark,Married/separated,0,58
1460,"al-Basa, Annnees",Male,1947-09-19,2200,Denmark,Married/separated,0,77
1472,"Roybal, Karla",Female,1952-03-25,2400,Denmark,Married/separated,0,72


In [6]:
matched_register['citizenship'] = matched_register['citizenship'].apply(
    lambda x: 'Denmark' if x == 'Denmark' else 'Other'
)
marital_mapping = {
    'Never married': 'Not in relationship',
    'Divorced': 'Not in relationship',
    'Married/separated': 'In relationship',
    'Widowed': 'Not in relationship'
}
matched_register['marital_status'] = matched_register['marital_status'].map(marital_mapping)


/tmp/ipykernel_29099/1594107926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_register['citizenship'] = matched_register['citizenship'].apply(
/tmp/ipykernel_29099/1594107926.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_register['marital_status'] = matched_register['marital_status'].map(marital_mapping)


In [7]:
matched_register

,name,sex,dob,zip,citizenship,marital_status,last_voted,age
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Not in relationship,1,20
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Not in relationship,1,23
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Not in relationship,1,22
9,"Grant, Debray",Male,2003-12-03,2100,Denmark,Not in relationship,1,20
10,"Blackmore, Gabriela",Female,2004-09-18,2100,Denmark,Not in relationship,1,20
...,...,...,...,...,...,...,...,...
1447,"el-Shahin, Muzaina",Female,1926-11-02,2300,Denmark,Not in relationship,0,98
1454,"Herrera, Addison",Female,1966-07-05,2300,Denmark,In relationship,0,58
1460,"al-Basa, Annnees",Male,1947-09-19,2200,Denmark,In relationship,0,77
1472,"Roybal, Karla",Female,1952-03-25,2400,Denmark,In relationship,0,72


In [8]:
indexer = recordlinkage.Index()
indexer.full()
#indexer.block('sex')
#indexer.block('citizenship')
#indexer.block('marital_status')
# indexer.block('evote')
# indexer.block('age')

candidate_links = indexer.index(matched_register, df_anonymised)
candidate_links

MultiIndex([(   1,   0),
            (   1,   1),
            (   1,   2),
            (   1,   3),
            (   1,   4),
            (   1,   5),
            (   1,   6),
            (   1,   7),
            (   1,   8),
            (   1,   9),
            ...
            (1475, 186),
            (1475, 187),
            (1475, 188),
            (1475, 189),
            (1475, 190),
            (1475, 191),
            (1475, 192),
            (1475, 193),
            (1475, 194),
            (1475, 195)],
           length=39004)

In [9]:
compare = recordlinkage.Compare()

compare.exact('sex', 'sex')
compare.exact('citizenship', 'citizenship')
compare.exact('marital_status', 'marital_status')
compare.exact('last_voted', 'evote')
#compare.numeric('age', 'age')

# The comparison vectors
compare_vectors = compare.compute(candidate_links, matched_register, df_anonymised)

In [10]:
print(compare_vectors)

          0  1  2  3
1    0    0  0  0  0
     1    1  1  0  0
     2    0  1  0  0
     3    1  1  0  1
     4    1  1  1  0
...      .. .. .. ..
1475 191  0  1  0  1
     192  0  1  1  0
     193  1  1  1  1
     194  1  1  0  1
     195  0  1  0  0

[39004 rows x 4 columns]


In [11]:
matches = compare_vectors[compare_vectors.sum(axis=1) >= 4]
matches

0  1  2  3
1    10   1  1  1  1
     17   1  1  1  1
     22   1  1  1  1
     25   1  1  1  1
     29   1  1  1  1
...      .. .. .. ..
1475 169  1  1  1  1
     180  1  1  1  1
     182  1  1  1  1
     184  1  1  1  1
     193  1  1  1  1

[5048 rows x 4 columns]

In [ ]:
# Save matches to CSV
matches.to_csv('matches.csv')
match_df = pd.read_csv('matches.csv')
match_df.columns = ['public_idx', 'an_idx', '0', '1', '2', '3']
match_df.head()

,public_idx,an_idx,0,1,2,3
0,1,10,1,1,1,1
1,1,17,1,1,1,1
2,1,22,1,1,1,1
3,1,25,1,1,1,1
4,1,29,1,1,1,1
...,...,...,...,...,...,...
5043,1475,169,1,1,1,1
5044,1475,180,1,1,1,1
5045,1475,182,1,1,1,1
5046,1475,184,1,1,1,1


In [13]:
grouped = match_df.groupby('public_idx')

results = {}

for public_idx, group in grouped:
  name = df_register.loc[public_idx, 'name']

  anonymised_indices = group['an_idx']
  parties = df_anonymised.loc[anonymised_indices, 'party'].tolist()

  if len(parties) == 1:
    party = parties[0]
    results[name] = party
  else:
    if len(set(parties)) == 1:
      party = parties[0]
      results[name] = party

results_df = pd.DataFrame(list(results.items()), columns=['name', 'party'])
print(results_df)

results_df.to_csv('final_matches.csv')

              name  party
0  Aguilar, Damien    Red
1    Walker, Natae  Green
2  Pierce, Andejia  Green
3      Quinn, Luke  Green
